# Análise Avançada de Explicabilidade com SHAP

Este notebook apresenta técnicas avançadas de explicabilidade para o modelo de recomendação utilizando SHAP (SHapley Additive exPlanations).

Vamos explorar:
1. Visualizações avançadas de SHAP
2. Análise de dependência de features
3. Explicabilidade por grupos de clientes
4. Comparação de explicações entre diferentes modelos
5. Geração de relatórios personalizados

In [ ]:
# Importações necessárias
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from io import BytesIO
import matplotlib
matplotlib.use('Agg')

# Adicionar diretório raiz ao path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Importar funções personalizadas
from src.config.model_utils import create_model_with_regularization
from src.mlflow_config import setup_mlflow, log_model_metrics, load_registered_model

In [ ]:
# Configurar MLflow
experiment_id = setup_mlflow(experiment_name="shap_analysis")

## 1. Carregamento e Preparação dos Dados

In [ ]:
# Carregar dados
dados = pd.read_parquet('../dados/dados_tratados.parquet')
print(f"Dados carregados: {dados.shape}")
dados.head()

In [ ]:
# Preparar dados para o modelo
def preparar_dados(dados):
    # Codificar variáveis categóricas
    customer_encoder = LabelEncoder()
    product_encoder = LabelEncoder()
    category_encoder = LabelEncoder()
    subcategory_encoder = LabelEncoder()
    
    # Aplicar encoding
    customer_ids = customer_encoder.fit_transform(dados['customer_id'])
    product_ids = product_encoder.fit_transform(dados['product_id'])
    category_ids = category_encoder.fit_transform(dados['category'])
    subcategory_ids = subcategory_encoder.fit_transform(dados['subcategory'])
    
    # Target
    sales = dados['sales'].values
    
    # Dimensões para embeddings
    num_customers = len(customer_encoder.classes_)
    num_products = len(product_encoder.classes_)
    num_categories = len(category_encoder.classes_)
    num_subcategories = len(subcategory_encoder.classes_)
    
    print(f"Número de clientes únicos: {num_customers}")
    print(f"Número de produtos únicos: {num_products}")
    print(f"Número de categorias únicas: {num_categories}")
    print(f"Número de subcategorias únicas: {num_subcategories}")
    
    # Dividir em treino e teste
    X = (customer_ids, product_ids, category_ids, subcategory_ids)
    y = sales
    
    X_train, X_test, y_train, y_test = train_test_split(
        list(zip(customer_ids, product_ids, category_ids, subcategory_ids)),
        y,
        test_size=0.2,
        random_state=42
    )
    
    # Descompactar X_train e X_test
    customer_train, product_train, category_train, subcategory_train = zip(*X_train)
    customer_test, product_test, category_test, subcategory_test = zip(*X_test)
    
    # Converter para arrays numpy
    customer_train = np.array(customer_train)
    product_train = np.array(product_train)
    category_train = np.array(category_train)
    subcategory_train = np.array(subcategory_train)
    
    customer_test = np.array(customer_test)
    product_test = np.array(product_test)
    category_test = np.array(category_test)
    subcategory_test = np.array(subcategory_test)
    
    # Retornar dados preparados
    return {
        'X_train': [customer_train, product_train, category_train, subcategory_train],
        'y_train': y_train,
        'X_test': [customer_test, product_test, category_test, subcategory_test],
        'y_test': y_test,
        'encoders': {
            'customer': customer_encoder,
            'product': product_encoder,
            'category': category_encoder,
            'subcategory': subcategory_encoder
        },
        'dims': {
            'num_customers': num_customers,
            'num_products': num_products,
            'num_categories': num_categories,
            'num_subcategories': num_subcategories
        },
        'dados_originais': dados
    }

# Preparar dados
dados_prep = preparar_dados(dados)

## 2. Carregar ou Treinar Modelo

In [ ]:
# Tentar carregar modelo registrado
model = load_registered_model(model_name="recommendation_model", stage="Production")

# Se não encontrar modelo registrado, treinar um novo
if model is None:
    print("Modelo registrado não encontrado. Treinando novo modelo...")
    
    # Criar modelo com regularização
    model = create_model_with_regularization(
        num_customers=dados_prep['dims']['num_customers'],
        num_products=dados_prep['dims']['num_products'],
        num_categories=dados_prep['dims']['num_categories'],
        num_subcategories=dados_prep['dims']['num_subcategories'],
        embedding_dim=50,
        l2_strength=0.001,
        dropout_rate=0.2
    )
    
    # Treinar modelo
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    
    history = model.fit(
        dados_prep['X_train'],
        dados_prep['y_train'],
        epochs=20,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Avaliar modelo
    loss = model.evaluate(dados_prep['X_test'], dados_prep['y_test'], verbose=0)
    print(f"Teste MSE: {loss}")
    
    # Registrar modelo no MLflow
    metrics = {"mse": loss}
    params = {
        "embedding_dim": 50,
        "l2_strength": 0.001,
        "dropout_rate": 0.2
    }
    
    run_id = log_model_metrics(model, metrics, params)
    print(f"Modelo registrado com run_id: {run_id}")
else:
    print("Modelo carregado do registro MLflow.")

## 3. Funções Avançadas de SHAP

In [ ]:
def create_shap_explainer(model, background_data, nsamples=500):
    """
    Cria um explainer SHAP para o modelo de recomendação.
    
    Args:
        model: Modelo treinado
        background_data: Dados de background para o explainer
        nsamples: Número de amostras para o KernelExplainer
        
    Returns:
        Explainer SHAP
    """
    # Função para prever com o modelo
    def model_predict(X):
        # Extrair features do formato esperado pelo KernelExplainer
        batch_size = X.shape[0]
        customer_ids = X[:, 0].astype(int)
        product_ids = X[:, 1].astype(int)
        category_ids = X[:, 2].astype(int)
        subcategory_ids = X[:, 3].astype(int)
        
        # Prever
        return model.predict(
            [customer_ids, product_ids, category_ids, subcategory_ids],
            verbose=0
        ).flatten()
    
    # Criar explainer
    explainer = shap.KernelExplainer(model_predict, background_data, nsamples=nsamples)
    return explainer

def prepare_shap_data(X_test, num_samples=100):
    """
    Prepara dados para análise SHAP.
    
    Args:
        X_test: Dados de teste [customer_ids, product_ids, category_ids, subcategory_ids]
        num_samples: Número de amostras a selecionar
        
    Returns:
        Dados formatados para SHAP
    """
    # Selecionar amostras aleatórias
    indices = np.random.choice(len(X_test[0]), min(num_samples, len(X_test[0])), replace=False)
    
    # Extrair dados
    customer_ids = X_test[0][indices]
    product_ids = X_test[1][indices]
    category_ids = X_test[2][indices]
    subcategory_ids = X_test[3][indices]
    
    # Combinar em um array
    X_shap = np.column_stack([customer_ids, product_ids, category_ids, subcategory_ids])
    
    return X_shap, indices

def get_feature_names(dados_prep):
    """
    Obtém nomes das features para visualizações SHAP.
    
    Args:
        dados_prep: Dados preparados
        
    Returns:
        Lista de nomes de features
    """
    return ["Cliente", "Produto", "Categoria", "Subcategoria"]

## 4. Preparar Dados para SHAP

In [ ]:
# Preparar dados para SHAP
background_data, _ = prepare_shap_data(dados_prep['X_train'], num_samples=200)
test_data, test_indices = prepare_shap_data(dados_prep['X_test'], num_samples=100)

# Criar explainer
explainer = create_shap_explainer(model, background_data, nsamples=500)

# Calcular valores SHAP
shap_values = explainer.shap_values(test_data)

# Obter nomes das features
feature_names = get_feature_names(dados_prep)

## 5. Visualizações Avançadas de SHAP

In [ ]:
# 5.1 Summary Plot
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, test_data, feature_names=feature_names, show=False)
plt.title("Impacto das Features nas Recomendações", fontsize=14)
plt.tight_layout()
plt.savefig("../reports/figuras/shap_summary_plot.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 5.2 Bar Plot
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, test_data, feature_names=feature_names, plot_type="bar", show=False)
plt.title("Importância Global das Features", fontsize=14)
plt.tight_layout()
plt.savefig("../reports/figuras/shap_bar_plot.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 5.3 Dependence Plot para cada feature
for i, feature in enumerate(feature_names):
    plt.figure(figsize=(10, 6))
    shap.dependence_plot(i, shap_values, test_data, feature_names=feature_names, show=False)
    plt.title(f"Dependence Plot: {feature}", fontsize=14)
    plt.tight_layout()
    plt.savefig(f"../reports/figuras/shap_dependence_{feature}.png", dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# 5.4 Force Plot para uma instância específica
# Selecionar uma instância aleatória
instance_idx = np.random.randint(0, len(test_data))
instance = test_data[instance_idx:instance_idx+1]

# Obter cliente e produto originais
customer_id = dados_prep['encoders']['customer'].inverse_transform([int(instance[0, 0])])[0]
product_id = dados_prep['encoders']['product'].inverse_transform([int(instance[0, 1])])[0]
category = dados_prep['encoders']['category'].inverse_transform([int(instance[0, 2])])[0]
subcategory = dados_prep['encoders']['subcategory'].inverse_transform([int(instance[0, 3])])[0]

print(f"Cliente: {customer_id}")
print(f"Produto: {product_id}")
print(f"Categoria: {category}")
print(f"Subcategoria: {subcategory}")

# Criar force plot
plt.figure(figsize=(20, 3))
force_plot = shap.force_plot(explainer.expected_value, shap_values[instance_idx], instance[0], 
                             feature_names=feature_names, matplotlib=True, show=False)
plt.title(f"Explicação da Recomendação para Cliente {customer_id}, Produto {product_id}", fontsize=14)
plt.tight_layout()
plt.savefig(f"../reports/figuras/shap_force_plot_{customer_id}_{product_id}.png", dpi=300, bbox_inches='tight')
plt.show()

## 6. Análise por Grupos de Clientes

In [ ]:
# Agrupar clientes por volume de compras
def analisar_grupos_clientes(dados_prep, explainer, model):
    """
    Analisa explicabilidade por grupos de clientes.
    """
    # Calcular volume de compras por cliente
    dados = dados_prep['dados_originais']
    volume_por_cliente = dados.groupby('customer_id')['sales'].sum().reset_index()
    
    # Definir quartis
    quartis = volume_por_cliente['sales'].quantile([0.25, 0.5, 0.75]).values
    
    # Categorizar clientes
    def categorizar_cliente(volume):
        if volume <= quartis[0]:
            return "Baixo Volume"
        elif volume <= quartis[1]:
            return "Médio-Baixo Volume"
        elif volume <= quartis[2]:
            return "Médio-Alto Volume"
        else:
            return "Alto Volume"
    
    volume_por_cliente['grupo'] = volume_por_cliente['sales'].apply(categorizar_cliente)
    
    # Mesclar com dados originais
    dados_com_grupo = dados.merge(volume_por_cliente[['customer_id', 'grupo']], on='customer_id')
    
    # Analisar cada grupo
    grupos = ['Baixo Volume', 'Médio-Baixo Volume', 'Médio-Alto Volume', 'Alto Volume']
    resultados = {}
    
    for grupo in grupos:
        print(f"\nAnalisando grupo: {grupo}")
        
        # Filtrar dados do grupo
        dados_grupo = dados_com_grupo[dados_com_grupo['grupo'] == grupo]
        
        # Selecionar amostra aleatória
        if len(dados_grupo) > 50:
            dados_grupo = dados_grupo.sample(50, random_state=42)
        
        # Preparar dados para SHAP
        customer_ids = dados_prep['encoders']['customer'].transform(dados_grupo['customer_id'])
        product_ids = dados_prep['encoders']['product'].transform(dados_grupo['product_id'])
        category_ids = dados_prep['encoders']['category'].transform(dados_grupo['category'])
        subcategory_ids = dados_prep['encoders']['subcategory'].transform(dados_grupo['subcategory'])
        
        X_grupo = np.column_stack([customer_ids, product_ids, category_ids, subcategory_ids])
        
        # Calcular valores SHAP
        shap_values_grupo = explainer.shap_values(X_grupo)
        
        # Calcular importância média das features
        importancia_media = np.abs(shap_values_grupo).mean(axis=0)
        
        # Armazenar resultados
        resultados[grupo] = {
            'shap_values': shap_values_grupo,
            'X': X_grupo,
            'importancia_media': importancia_media
        }
        
        # Mostrar importância média
        print("Importância média das features:")
        for i, feature in enumerate(feature_names):
            print(f"{feature}: {importancia_media[i]:.4f}")
    
    # Visualizar comparação entre grupos
    plt.figure(figsize=(12, 8))
    
    # Preparar dados para o gráfico
    importancias = []
    for grupo in grupos:
        importancias.append(resultados[grupo]['importancia_media'])
    
    importancias = np.array(importancias)
    
    # Criar gráfico de barras agrupadas
    x = np.arange(len(feature_names))
    width = 0.2
    
    for i, grupo in enumerate(grupos):
        plt.bar(x + i*width - 0.3, importancias[i], width, label=grupo)
    
    plt.xlabel('Features')
    plt.ylabel('Importância SHAP Média')
    plt.title('Comparação da Importância das Features por Grupo de Clientes')
    plt.xticks(x, feature_names)
    plt.legend()
    plt.tight_layout()
    plt.savefig("../reports/figuras/shap_comparacao_grupos.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    return resultados

# Executar análise por grupos
resultados_grupos = analisar_grupos_clientes(dados_prep, explainer, model)

## 7. Geração de Relatório Personalizado Avançado

In [ ]:
def gerar_relatorio_personalizado(cliente_id, model, explainer, dados_prep, num_recomendacoes=5):
    """
    Gera um relatório personalizado com recomendações e explicações SHAP para um cliente específico.
    
    Args:
        cliente_id: ID do cliente
        model: Modelo treinado
        explainer: Explainer SHAP
        dados_prep: Dados preparados
        num_recomendacoes: Número de recomendações a gerar
    """
    # Verificar se o cliente existe
    dados = dados_prep['dados_originais']
    if cliente_id not in dados['customer_id'].unique():
        print(f"Cliente {cliente_id} não encontrado nos dados.")
        return
    
    # Obter dados do cliente
    dados_cliente = dados[dados['customer_id'] == cliente_id]
    
    # Obter produtos já comprados pelo cliente
    produtos_comprados = set(dados_cliente['product_id'].unique())
    
    # Obter todos os produtos disponíveis
    todos_produtos = set(dados['product_id'].unique())
    
    # Produtos que o cliente ainda não comprou
    produtos_nao_comprados = list(todos_produtos - produtos_comprados)
    
    # Preparar dados para predição
    customer_id_encoded = dados_prep['encoders']['customer'].transform([cliente_id])[0]
    product_ids_encoded = dados_prep['encoders']['product'].transform(produtos_nao_comprados)
    
    # Obter categorias e subcategorias para cada produto
    categorias = []
    subcategorias = []
    
    for produto in produtos_nao_comprados:
        produto_info = dados[dados['product_id'] == produto].iloc[0]
        categorias.append(produto_info['category'])
        subcategorias.append(produto_info['subcategory'])
    
    category_ids_encoded = dados_prep['encoders']['category'].transform(categorias)
    subcategory_ids_encoded = dados_prep['encoders']['subcategory'].transform(subcategorias)
    
    # Criar arrays para predição
    customer_array = np.full(len(produtos_nao_comprados), customer_id_encoded)
    
    # Prever scores
    scores = model.predict(
        [customer_array, product_ids_encoded, category_ids_encoded, subcategory_ids_encoded],
        verbose=0
    ).flatten()
    
    # Obter top-k produtos recomendados
    top_indices = np.argsort(scores)[::-1][:num_recomendacoes]
    top_produtos = [produtos_nao_comprados[i] for i in top_indices]
    top_scores = scores[top_indices]
    top_categorias = [categorias[i] for i in top_indices]
    top_subcategorias = [subcategorias[i] for i in top_indices]
    
    # Preparar dados para SHAP
    X_shap = []
    for i in range(len(top_produtos)):
        X_shap.append([
            customer_id_encoded,
            dados_prep['encoders']['product'].transform([top_produtos[i]])[0],
            dados_prep['encoders']['category'].transform([top_categorias[i]])[0],
            dados_prep['encoders']['subcategory'].transform([top_subcategorias[i]])[0]
        ])
    
    X_shap = np.array(X_shap)
    
    # Calcular valores SHAP
    shap_values = explainer.shap_values(X_shap)
    
    # Criar PDF
    pdf_path = f"../reports/relatorio_cliente_{cliente_id}.pdf"
    doc = SimpleDocTemplate(pdf_path, pagesize=letter)
    styles = getSampleStyleSheet()
    elements = []
    
    # Título
    title_style = styles['Title']
    elements.append(Paragraph(f"Relatório de Recomendações Personalizado", title_style))
    elements.append(Spacer(1, 12))
    
    # Informações do cliente
    elements.append(Paragraph(f"Cliente: {cliente_id}", styles['Heading2']))
    elements.append(Spacer(1, 12))
    
    # Histórico de compras
    elements.append(Paragraph("Histórico de Compras", styles['Heading2']))
    elements.append(Spacer(1, 6))
    
    # Tabela com histórico de compras
    historico = dados_cliente.sort_values('sales', ascending=False).head(10)
    data = [["Produto", "Categoria", "Subcategoria", "Vendas"]]
    for _, row in historico.iterrows():
        data.append([row['product_id'], row['category'], row['subcategory'], f"{row['sales']:.2f}"])
    
    table = Table(data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(table)
    elements.append(Spacer(1, 12))
    
    # Recomendações
    elements.append(Paragraph("Produtos Recomendados", styles['Heading2']))
    elements.append(Spacer(1, 6))
    
    # Para cada recomendação
    for i in range(len(top_produtos)):
        # Informações do produto
        elements.append(Paragraph(f"Recomendação #{i+1}: {top_produtos[i]}", styles['Heading3']))
        elements.append(Paragraph(f"Categoria: {top_categorias[i]}", styles['Normal']))
        elements.append(Paragraph(f"Subcategoria: {top_subcategorias[i]}", styles['Normal']))
        elements.append(Paragraph(f"Score: {top_scores[i]:.4f}", styles['Normal']))
        elements.append(Spacer(1, 6))
        
        # Gerar force plot
        plt.figure(figsize=(10, 2))
        shap.force_plot(
            explainer.expected_value, 
            shap_values[i], 
            X_shap[i], 
            feature_names=feature_names,
            matplotlib=True,
            show=False
        )
        plt.title(f"Explicação da Recomendação")
        plt.tight_layout()
        
        # Salvar figura em buffer
        img_buffer = BytesIO()
        plt.savefig(img_buffer, format='png', dpi=150, bbox_inches='tight')
        plt.close()
        
        # Adicionar imagem ao PDF
        img_buffer.seek(0)
        img = Image(img_buffer, width=450, height=100)
        elements.append(img)
        elements.append(Spacer(1, 12))
        
        # Explicação textual
        elements.append(Paragraph("Explicação:", styles['Heading4']))
        
        # Ordenar features por importância
        feature_importance = [(feature_names[j], abs(shap_values[i][j])) for j in range(len(feature_names))]
        feature_importance.sort(key=lambda x: x[1], reverse=True)
        
        for feature, importance in feature_importance:
            direction = "aumenta" if shap_values[i][feature_names.index(feature)] > 0 else "diminui"
            elements.append(Paragraph(
                f"• {feature}: {direction} a probabilidade de recomendação (impacto: {importance:.4f})",
                styles['Normal']
            ))
        
        elements.append(Spacer(1, 12))
    
    # Resumo geral
    elements.append(Paragraph("Resumo da Análise", styles['Heading2']))
    elements.append(Spacer(1, 6))
    
    # Gerar bar plot com importância média das features
    plt.figure(figsize=(8, 5))
    importancia_media = np.abs(shap_values).mean(axis=0)
    plt.bar(feature_names, importancia_media)
    plt.xlabel('Features')
    plt.ylabel('Importância SHAP Média')
    plt.title('Importância Média das Features nas Recomendações')
    plt.tight_layout()
    
    # Salvar figura em buffer
    img_buffer = BytesIO()
    plt.savefig(img_buffer, format='png', dpi=150, bbox_inches='tight')
    plt.close()
    
    # Adicionar imagem ao PDF
    img_buffer.seek(0)
    img = Image(img_buffer, width=400, height=250)
    elements.append(img)
    elements.append(Spacer(1, 12))
    
    # Conclusão
    elements.append(Paragraph("Conclusão", styles['Heading2']))
    elements.append(Spacer(1, 6))
    
    # Determinar feature mais importante
    feature_mais_importante = feature_names[np.argmax(importancia_media)]
    
    elements.append(Paragraph(
        f"Com base na análise SHAP, a feature '{feature_mais_importante}' tem o maior impacto nas recomendações "
        f"para este cliente. As recomendações foram geradas considerando o histórico de compras do cliente "
        f"e padrões identificados pelo modelo de recomendação.",
        styles['Normal']
    ))
    
    # Construir PDF
    doc.build(elements)
    
    print(f"Relatório gerado com sucesso: {pdf_path}")
    return pdf_path

# Gerar relatório para um cliente específico
# Selecionar um cliente aleatório
cliente_aleatorio = np.random.choice(dados['customer_id'].unique())
relatorio_path = gerar_relatorio_personalizado(cliente_aleatorio, model, explainer, dados_prep)

## 8. Comparação de Modelos com SHAP

In [ ]:
def comparar_modelos_com_shap(model_original, model_regularizado, dados_prep, test_data):
    """
    Compara dois modelos usando SHAP.
    """
    # Preparar dados de background
    background_data, _ = prepare_shap_data(dados_prep['X_train'], num_samples=200)
    
    # Criar explainers
    explainer_original = create_shap_explainer(model_original, background_data, nsamples=300)
    explainer_regularizado = create_shap_explainer(model_regularizado, background_data, nsamples=300)
    
    # Calcular valores SHAP
    shap_values_original = explainer_original.shap_values(test_data)
    shap_values_regularizado = explainer_regularizado.shap_values(test_data)
    
    # Calcular importância média das features
    importancia_original = np.abs(shap_values_original).mean(axis=0)
    importancia_regularizado = np.abs(shap_values_regularizado).mean(axis=0)
    
    # Visualizar comparação
    plt.figure(figsize=(12, 6))
    
    x = np.arange(len(feature_names))
    width = 0.35
    
    plt.bar(x - width/2, importancia_original, width, label='Modelo Original')
    plt.bar(x + width/2, importancia_regularizado, width, label='Modelo Regularizado')
    
    plt.xlabel('Features')
    plt.ylabel('Importância SHAP Média')
    plt.title('Comparação da Importância das Features entre Modelos')
    plt.xticks(x, feature_names)
    plt.legend()
    plt.tight_layout()
    plt.savefig("../reports/figuras/shap_comparacao_modelos.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    # Calcular diferença de importância
    diferenca = importancia_regularizado - importancia_original
    
    plt.figure(figsize=(10, 6))
    plt.bar(feature_names, diferenca, color=['green' if d > 0 else 'red' for d in diferenca])
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    plt.xlabel('Features')
    plt.ylabel('Diferença de Importância (Regularizado - Original)')
    plt.title('Mudança na Importância das Features após Regularização')
    plt.tight_layout()
    plt.savefig("../reports/figuras/shap_diferenca_modelos.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    return {
        'importancia_original': importancia_original,
        'importancia_regularizado': importancia_regularizado,
        'diferenca': diferenca
    }

# Nota: Para executar esta função, seria necessário ter dois modelos diferentes para comparação.
# Como exemplo, podemos criar um modelo sem regularização para comparar com o modelo regularizado.
# Este código está comentado pois depende de ter dois modelos disponíveis.

# model_original = load_registered_model(model_name="recommendation_model_original", stage="Production")
# if model_original:
#     resultados_comparacao = comparar_modelos_com_shap(model_original, model, dados_prep, test_data)

## 9. Conclusão e Próximos Passos

Neste notebook, exploramos técnicas avançadas de explicabilidade para o modelo de recomendação utilizando SHAP. As principais conclusões são:

1. Visualizações avançadas de SHAP permitem entender melhor o comportamento do modelo
2. A análise por grupos de clientes revela diferenças na importância das features para diferentes perfis
3. Os relatórios personalizados fornecem insights detalhados sobre as recomendações para cada cliente
4. A comparação entre modelos ajuda a entender o impacto da regularização na interpretabilidade

Próximos passos:

1. Integrar estas análises ao pipeline de MLOps
2. Automatizar a geração de relatórios para clientes importantes
3. Explorar técnicas adicionais de explicabilidade, como LIME ou Integrated Gradients
4. Desenvolver um dashboard interativo para visualização das explicações SHAP